In [ ]:
import serial
import time
from datetime import date
from openpyxl import load_workbook
import openpyxl

# Load the Excel file
workbook = load_workbook(r"C:\Users\Bharani\OneDrive\Documents\attendance_system.xlsx")
worksheet = workbook.active

# Function to find the next column
def next_column(sheet):
    maxcol = sheet.max_column
    return openpyxl.utils.get_column_letter(maxcol + 1)

# Function to check if today's date column exists
def get_today_column(sheet, current_date):
    for col in range(1, sheet.max_column + 1):
        col_letter = openpyxl.utils.get_column_letter(col)
        if sheet[col_letter + "1"].value == current_date.strftime('%d-%m-%Y'):
            return col_letter
    return None

# Function to mark attendance
def mark_attendance(id_number):
    worksheet[nxt + str(id_number + 1)] = '/'
    workbook.save(r"C:\Users\Bharani\OneDrive\Documents\attendance_system.xlsx")
    rollnumber = worksheet['B' + str(id_number + 1)].value
    print('Attendance marked for {} '.format(rollnumber))
    return

# Get today's date
current_date = date.today()
head = current_date.strftime('%d-%m-%Y')

# Check for today's date column or create a new one
today_column = get_today_column(worksheet, current_date)
if today_column:
    nxt = today_column
else:
    nxt = str(next_column(worksheet))
    worksheet[nxt + "1"] = head

start_time = time.time()
ser = serial.Serial('COM10', 9600)

while True:
    if ser.in_waiting > 0:
        id_number = int(ser.readline().decode().strip())
        mark_attendance(id_number)

        elapsed_time = time.time() - start_time
        if elapsed_time >= 30:  
            print("30 minutes have elapsed. Attendance process is over")
            break  # Exit the loop
ser.close()
